In [1]:
using Flux.Tracker
using Flux
using Flux:throttle,glorot_uniform
using BSON:@save
using BSON:@load
using Base.Iterators: repeated
using Flux:@treelike
using PyPlot
using FastGaussQuadrature
using SparseGrids
using LinearAlgebra
using Flux: @epochs
using DataFrames
using CSV
using Sobol
using Pkg; Pkg.activate("cuda"); Pkg.instantiate()
using CuArrays
using ForwardDiff
using CUDAnative
CuArrays.culiteral_pow(::typeof(^), x::ForwardDiff.Dual{Nothing,Float32,1}, ::Val{2}) = x*x
CuArrays.culiteral_pow(::typeof(^), x::ForwardDiff.Dual{Nothing,Float64,1}, ::Val{2}) = x*x

f(x)=2*π^2 .*sum(cos.(π*x[i,:]') for i=1:8, dims=1)
acti(x)=@. x/(1+CUDAnative.exp(-x))


struct Block{F,S,T}
  W1::S
  W2::S
  b1::T
  b2::T
  σ::F
end

Block(W1 , W2, b1, b2) = Block(W1,W2 ,b1 ,b2, identity)

function Block(in1::Integer,in2::Integer, out1::Integer,out2::Integer, σ = identity;
  initW1 = glorot_uniform, initW2 =glorot_uniform, initb1 = zeros,initb2=zeros)
  return Block(param(initW1(out1, in1)),param(initW2(out2,in2)), param(initb1(out1)), param(initb2(out2)),σ)
end

@treelike Block

function (a::Block)(x)
  W1,W2, b1,b2, σ = a.W1,a.W2,a.b1, a.b2, a.σ
  σ.(W2 *σ.(W1*x .+ b1) .+ b2) .+ x
end





M=Chain(
Dense(8,30),
Block(30,30,30,30,acti),
Block(30,30,30,30,acti),
Block(30,30,30,30,acti),
Block(30,30,30,30,acti),
Dense(30,1)
)|>gpu

#=
@load "test1FD.bson" M
@load "test2FD.bson" weights
Flux.loadparams!(M, weights)
=#


#=

function loss(s,y=1)
  loss = sum((M(s) - 0.25*(s[1,:].^2+s[2,:].^2 .-1)').^2)
  return loss
end
=#

function lossFD(N)
  notes=rand(8,N)
  notes =notes |>gpu
  d=rand(1)/100 |>gpu
  losses=0
  MMM=M(notes)
  for i=1:8
    l1=zeros(8)
    l1[i]=1
    l1 = l1 |>gpu
    losses  += (sum(0.5*((M(notes .+ d.*l1) .- MMM)./d).^2)/N)[1]
  end
  losses += (sum(0.5*(π^2 * (MMM).^2))/N)[1]
  losses += (sum(-f(notes) .*MMM)/N)[1]
  return losses
end


function lossBC(β)
  d=(rand(1)/100)|>gpu
  point1=zeros(8,100)
    point2=zeros(8,100)
  p=rand(7,100)
    losses=0
  for i=1:8
    point1=vcat(p[1:i-1,:],zeros(100)',p[i:end,:])
    point2=vcat(p[1:i-1,:], ones(100)',p[i:end,:])
    point1=point1|>gpu
    point2=point2|>gpu
    l1=zeros(8)
    l1[i]=1
    l1=l1|>gpu
    losses +=β*(sum((( M(point1 .+ d.*l1) .- 1/2*M(point1 .- d.*l1))./d).^2)/100)[1]
    losses +=β*(sum((( M(point2 .+ d.*l1) .- 1/2*M(point2 .- d.*l1))./d).^2)/100)[1]
  end
  return losses
end

function loss1(N)
  return lossBC(50)+lossFD(N)
end



function loss_true(xx=1,yy=1,xx1=1,xx2=2)
  points=zeros(8,5000)
    for i=1:5000
      points[:,i]=next!(truep)
    end
    points=points  |>gpu
  F_true(x)= sum(cos.(pi*x[i,:]') for i=1:8, dims=1)
  errors=sqrt(sum((M(points)-F_true(points)).^2)/2000)
  return errors
end
truep=SobolSeq(8)

inner=SobolSeq(8)
BC=SobolSeq(7)
function test(N;traintime=2000)
  errdf=DataFrame(time=Int[],absolute_error=Float64[],loss_FD=Float64[])
  cntr=1
  evalcb = function()
    loss_true1=loss_true()
    loss_FD = lossFD(N)
    push!(errdf,[Tracker.data(cntr),Tracker.data(loss_true1),Tracker.data(loss_FD)])
    cntr+=1
    if cntr%500==0
      @show(cntr,loss_FD,loss_true1)
      CSV.write("$(N)rand8Dexm2.csv",errdf)
      @save "$(N)randpoint8Dexm2m.bson" M
      weights=Tracker.data.(params(M))
      @save "$(N)randpoint8Dexm2w.bson" weights
    end
  end
  θ=Flux.params(M)
  opt=ADAM()
  dataset=[(N) for i=1:traintime]
  Flux.train!(loss1, params(M), zip(dataset), opt, cb=evalcb)
  return errdf
end


Activating new environment at `~/exm2/cuda/Project.toml`
  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...


test (generic function with 1 method)

In [2]:
errdf=test(100,traintime=50000)
for i=1:4
    N=[500 1000 2000 10000]
    errdf=test(N[i],traintime=20000)
end

cntr = 500
loss_FD = -1.0334833986499155 (tracked)
loss_true1 = 3.1213908f0 (tracked)
cntr = 1000
loss_FD = -3.665176419698687 (tracked)
loss_true1 = 2.727162f0 (tracked)
cntr = 1500
loss_FD = -17.384311406709553 (tracked)
loss_true1 = 2.687261f0 (tracked)
cntr = 2000
loss_FD = -12.667850922948414 (tracked)
loss_true1 = 2.4379857f0 (tracked)
cntr = 2500
loss_FD = -28.20858685736234 (tracked)
loss_true1 = 1.9261231f0 (tracked)
cntr = 3000
loss_FD = -24.806866764812625 (tracked)
loss_true1 = 1.8953888f0 (tracked)
cntr = 3500
loss_FD = -23.958021999597342 (tracked)
loss_true1 = 1.8711232f0 (tracked)
cntr = 4000
loss_FD = -27.08314245547675 (tracked)
loss_true1 = 1.518401f0 (tracked)
cntr = 4500
loss_FD = -51.78928523441421 (tracked)
loss_true1 = 1.1454775f0 (tracked)
cntr = 5000
loss_FD = -37.13086549359623 (tracked)
loss_true1 = 1.1183121f0 (tracked)
cntr = 5500
loss_FD = -48.19839827745224 (tracked)
loss_true1 = 1.1270417f0 (tracked)
cntr = 6000
loss_FD = -27.377716163024118 (tracked)
l

cntr = 48000
loss_FD = -38.18963587153564 (tracked)
loss_true1 = 1.1098703f0 (tracked)
cntr = 48500
loss_FD = -47.47684761648671 (tracked)
loss_true1 = 1.1058433f0 (tracked)
cntr = 49000
loss_FD = -35.018733607124496 (tracked)
loss_true1 = 0.8523546f0 (tracked)
cntr = 49500
loss_FD = -26.785726090159272 (tracked)
loss_true1 = 0.17485142f0 (tracked)
cntr = 50000
loss_FD = -32.705917639309384 (tracked)
loss_true1 = 0.112648904f0 (tracked)
cntr = 500
loss_FD = -34.85849436341843 (tracked)
loss_true1 = 0.0782591f0 (tracked)
cntr = 1000
loss_FD = -44.791521147600584 (tracked)
loss_true1 = 0.11673364f0 (tracked)
cntr = 1500
loss_FD = -37.2454224651885 (tracked)
loss_true1 = 0.08255682f0 (tracked)
cntr = 2000
loss_FD = -42.21926058143181 (tracked)
loss_true1 = 0.12156087f0 (tracked)
cntr = 2500
loss_FD = -39.96010977452045 (tracked)
loss_true1 = 0.059835408f0 (tracked)
cntr = 3000
loss_FD = -39.177873851708604 (tracked)
loss_true1 = 0.058534957f0 (tracked)
cntr = 3500
loss_FD = -44.8876073108

cntr = 5000
loss_FD = -38.05588763685688 (tracked)
loss_true1 = 0.034506638f0 (tracked)
cntr = 5500
loss_FD = -41.186827958492245 (tracked)
loss_true1 = 0.051630337f0 (tracked)
cntr = 6000
loss_FD = -36.95198671533876 (tracked)
loss_true1 = 0.059897598f0 (tracked)
cntr = 6500
loss_FD = -38.27910969228169 (tracked)
loss_true1 = 0.047668673f0 (tracked)
cntr = 7000
loss_FD = -41.122718128310375 (tracked)
loss_true1 = 0.09356612f0 (tracked)
cntr = 7500
loss_FD = -41.55103548989351 (tracked)
loss_true1 = 0.04692028f0 (tracked)
cntr = 8000
loss_FD = -38.96240601706752 (tracked)
loss_true1 = 0.031139297f0 (tracked)
cntr = 8500
loss_FD = -40.61456542498783 (tracked)
loss_true1 = 0.045282356f0 (tracked)
cntr = 9000
loss_FD = -37.2331828077587 (tracked)
loss_true1 = 0.03825566f0 (tracked)
cntr = 9500
loss_FD = -37.087677557831334 (tracked)
loss_true1 = 0.03351256f0 (tracked)
cntr = 10000
loss_FD = -40.42201324215938 (tracked)
loss_true1 = 0.034777284f0 (tracked)
cntr = 10500
loss_FD = -38.309852